In [1]:
const source_name = "Recommendations";

In [2]:
username = "Fro116"
source = "MAL"
task = "all";

In [3]:
using DataFrames
import CSV
import NBInclude: @nbinclude
import Statistics: mean, quantile, cor
@nbinclude("../InferenceAlphas/Alpha.ipynb");

In [4]:
ENV["DATAFRAMES_COLUMNS"] = 300;
ENV["DATAFRAMES_ROWS"] = 300;

In [5]:
DEBUG = true;

In [6]:
if source == "MAL"
    sourcename = "MyAnimeList"
elseif source == "AniList"
    sourcename = "AniList"
elseif source == "XML"
    sourcename = "XML"
else
    @assert false
end
@info "Displaying recommendations for $username from $sourcename"

[ Info: 20230213 04:48:40 Displaying recommendations for Fro116 from MyAnimeList


# Merge recommendations with item metadata

In [7]:
function to_hyperlink(title, url)
    "<a href=\"$url\">$title</a>"
end

function get_hyperlink(title, links; source = "MAL")
    if source == "MAL"
        search = "myanimelist"
    elseif source == "AniList"
        search = "anilist"
    elseif source == "XML"
        search = "MAL"
    else
        @assert false
    end

    # try to return the preferred source
    parsed_links = eval(Meta.parse(replace(links, "'" => "\"")))
    for link in parsed_links
        if occursin(search, link)
            return to_hyperlink(title, link)
        end
    end

    # default to mal
    search = "myanimelist"
    parsed_links = eval(Meta.parse(replace(links, "'" => "\"")))
    for link in parsed_links
        if occursin(search, link)
            return to_hyperlink(title, link)
        end
    end

    return title
end

function parse_time(x)
    minutes = Int(round(x / 60))
    hours = Int(floor(minutes / 60))
    minutes = minutes % 60
    function time_string(x, suffix)
        if x == 0
            return ""
        else
            return "$(x)$(suffix) "
        end
    end
    time_string(hours, "h") * time_string(minutes, "m")
end

function get_anime()
    df = DataFrame(
        CSV.File("../../data/processed_data/anime.csv", ntasks = 1; stringtype = String),
    )
    df.title = get_hyperlink.(df.title, df.links; source = source)
    df.average_episode_duration = parse_time.(df.average_episode_duration)
    df
end;

In [8]:
function get_rec_df(task)
    anime = get_anime()
    anime_to_uid = DataFrame(CSV.File("../../data/processed_data/anime_to_uid.csv"))
    anime_to_uid = innerjoin(anime_to_uid, anime, on = "animeid" => "anime_id")

    get_alpha(x) = read_recommendee_alpha("$task/$x", "all").rating
    rating_df = DataFrame(
        "uid" => 0:num_items()-1,
        "explicit" => get_alpha("Explicit"),
        "implicit" => get_alpha("LinearImplicit"),
        "baseline_explicit" => get_alpha("ExplicitUserItemBiases"),
    )
    rating_df[:, "score"] = get_alpha("MLE.Ensemble")
    rating_df[:, "relative_explicit"] =
        rating_df[:, "explicit"] ./ rating_df[:, "baseline_explicit"]

    # if DEBUG
    #     df = get_recommendee_split("explicit")
    #     rss = sum((df.rating - rating_df.explicit[df.item]) .^ 2)
    #     tss = sum((df.rating .- mean(df.rating)) .^ 2)
    #     @info "Debug Info"
    #     @info "RMSE: $(sqrt(rss / length(df.rating)))"
    #     @info "R2: $(1 - rss / tss)"
    #     @info "Correlation: $(cor(df.rating, rating_df.explicit[df.item]))"
    # end

    # don't recommend shows that the user has already seen before
    rating_df[:, "seen"] .= false
    rating_df.seen[get_recommendee_split("implicit").item] .= true
    rating_df[:, "ptw"] .= false
    rating_df.ptw[get_recommendee_split("ptw").item] .= true


    # don't recommend shows related to shows they have seen before
    rating_df[:, "related"] .= false
    rating_df.related[read_recommendee_alpha("all/RelatedSeries", "all").rating.!=0] .= true
    rating_df[:, "recap"] .= false
    rating_df.recap[read_recommendee_alpha("all/RecapSeries", "all").rating.!=0] .= true

    rating_df.score = rating_df.score .- mean(filter(x -> !x.seen, rating_df).score)

    rec_df = innerjoin(anime_to_uid, rating_df, on = "uid")
    keepcols = [
        "animeid",
        "uid",
        "title",
        "medium",
        "num_episodes",
        "average_episode_duration",
        "start_date",
        "genres",
        "tags",
    ]
    for x in names(rating_df)
        if x ∉ keepcols
            push!(keepcols, x)
        end
    end
    rec_df = rec_df[:, keepcols]
end;

In [9]:
for (content, name) in
    [("explicit", "rated"), ("implicit", "watched"), ("ptw", "planned to watch")]
    n = length(get_recommendee_split(content).item)
    @info "$n items $name"
end

[ Info: 20230213 04:48:44 468 items rated
[ Info: 20230213 04:48:44 471 items watched
[ Info: 20230213 04:48:44 29 items planned to watch


# Create display filters

In [10]:
display_filter(f) = (df; fn = identity) -> filter(fn ∘ f, df)
inv(f) = (x...) -> f(x...; fn = !)
seen = display_filter(x -> x.seen)
related = display_filter(x -> x.related)
recap = display_filter(x -> x.recap)
ptw = display_filter(x -> x.ptw)
tv = display_filter(x -> x.medium in ["tv", "ona"])
head(n) = x -> first(x, n);
top(n) = x -> first(sort(x, :score, rev = true), n);

In [11]:
function display(df::DataFrame)
    df
    df = select(
        df,
        :title,
        :score,
        Not([
            :title,
            :score,
            :recap,
            :related,
            :ptw,
            :seen,
            :animeid,
            :uid,
            :baseline_explicit,
        ]),
    )
    if !DEBUG
        df = select(df, Not([:explicit, :implicit, :relative_explicit]))
    end
    headers = titlecase.(replace.(names(df), "_" => " "))
    headers = replace(
        headers,
        "Medium" => "Type",
        "Explicit" => "Rating",
        "Implicit" => "Watch Probability",
        "Relative Explicit" => "Relative Rating",
        "Relative Implicit" => "Relative Watch Probability",
        "Average Episode Duration" => "Episode Length",
        "Num Episodes" => "Episodes",
    )
    Base.show(
        stdout,
        MIME("text/html"),
        df;
        allow_html_in_cells = true,
        header = headers,
        nosubheader = true,
        row_number_column_title = "Rank",
        top_left_str = "",
    )
end;

In [12]:
# curated_feed(n) = rec_df |> inv(seen) |> inv(related) |> outlier_filter(n, 0.1) |> display
# outlier_feed(n) =
#     rec_df |> inv(seen) |> inv(related) |> inv(outlier_filter(n, 0.1)) |> display
# continue_watching_tv(n) = rec_df |> tv |> related |> inv(seen) |> top(n) |> display
# continue_watching_specials(n) =
#     rec_df |> inv(tv) |> related |> inv(seen) |> top(n) |> display
# plan_to_watch(n) = rec_df |> ptw |> top(n) |> display;

In [17]:
N = 25;

In [18]:
get_rec_df("random") |> inv(seen) |> inv(related) |> top(N) |> display

Rank,Title,Score,Type,Episodes,Episode Length,Start Date,Genres,Tags,Rating,Watch Probability,Relative Rating
1,Hunter x Hunter (2011),6.95798,tv,148,24m,2011-10-02,"['Action', 'Adventure', 'Fantasy', 'Shounen']","['action', 'adventure', 'alternative present', 'alternative world', 'animals', 'anti-hero', 'assassins', 'based on a manga', 'bounty hunter', 'card battle', 'child protagonists', 'chimera', 'comedy', 'coming of age', 'crime', 'dark fantasy', 'drama', 'ensemble cast', 'environmental', 'excessive violence', 'fantasy', 'fantasy world', 'fighting-shounen', 'found family', 'ganbatte', 'gangs', 'gender bending', 'genius', 'gore', 'hero of strong character', 'lgbtq+ themes', 'mafia', 'magic', 'male protagonist', 'manga', 'martial arts', 'monster', 'monsters', 'philosophy', 'plot continuity', 'politics', 'primarily male cast', 'primarily teen cast', 'psi-powers', 'revenge', 'shogi', 'shounen', 'slapstick', 'splatter', 'super power', 'superpower', 'superpowers', 'tragedy', 'transgender', 'travel', 'tsundere', 'urban fantasy', 'video games', 'violence', 'virtual world', 'vore']",7.36143,0.00282985,0.982693
2,Kimagure Orange☆Road: Ano Hi ni Kaeritai,6.86513,movie,1,1h 9m,1988-10-01,"['Comedy', 'Drama', 'Romance', 'Shounen', 'Slice of Life']","['acting', 'asia', 'based on a manga', 'comedy', 'coming of age', 'drama', 'earth', 'heterosexual', 'japan', 'love polygon', 'love triangle', 'male protagonist', 'manga', 'photography', 'plot continuity', 'present', 'primarily teen cast', 'psychological drama', 'romance', 'romantic drama', 'school', 'seinen', 'shounen', 'slice of life', 'tragedy', 'urban']",7.86272,0.000108568,1.184
3,Cross Game,6.85759,tv,50,24m,2009-04-05,"['Drama', 'Romance', 'School', 'Shounen', 'Sports', 'Team Sports']","['asia', 'baseball', 'based on a manga', 'childhood friends', 'comedy', 'coming of age', 'daily life', 'death of a loved one', 'drama', 'female protagonist', 'ganbatte', 'hero of strong character', 'heterosexual', 'high school', 'japan', 'love triangle', 'male protagonist', 'manga', 'present', 'primarily teen cast', 'romance', 'school', 'school club', 'school clubs', 'school life', 'shounen', 'slice of life', 'sports', 'team sports', 'time skip', 'tomboy', 'tragedy', 'tsundere']",7.61861,0.00389741,1.07722
4,Full Moon wo Sagashite,6.80818,tv,52,24m,2002-04-06,"['Comedy', 'Drama', 'Music', 'Romance', 'Shoujo', 'Supernatural']","['age gap', 'age transformation', 'angels', 'asia', 'based on a manga', 'comedy', 'contemporary fantasy', 'drama', 'earth', 'female protagonist', 'henshin', 'heterosexual', 'idol', 'idols', 'illness', 'japan', 'love triangle', 'magic', 'magical girl', 'mahou shoujo', 'manga', 'music', 'musical', 'present', 'primarily child cast', 'rehabilitation', 'romance', 'school', 'sentimental drama', 'shinigami', 'shoujo', 'showbiz', 'supernatural', 'tragedy', 'work life', 'youkai']",7.26253,0.00128522,1.15159
5,Hikaru no Go,6.67416,tv,75,24m,2001-10-10,"['Comedy', 'Drama', 'School', 'Shounen', 'Strategy Game', 'Supernatural']","['asia', 'based on a manga', 'board games', 'body sharing', 'brain games and gambling', 'comedy', 'coming of age', 'drama', 'foreign', 'game', 'ganbatte', 'ghost', 'ghosts', 'go', 'japan', 'male protagonist', 'manga', 'plot continuity', 'present', 'psychological', 'school club', 'shounen', 'sports', 'strategy game', 'supernatural', 'tournaments']",7.30429,0.000593346,1.097
6,Kanon (2006),6.63909,tv,24,24m,2006-10-06,"['Drama', 'Romance', 'Slice of Life', 'Supernatural']","['age regression', 'amnesia', 'asia', 'based on a visual novel', 'comedy', 'coming of age', 'contemporary fantasy', 'demons', 'drama', 'earth', 'erotic game', 'family life', 'fantasy', 'female harem', 'game', 'ghost', 'harem', 'heterosexual', 'high school', 'japan', 'kuudere', 'male protagonist', 'moe', 'present', 'primarily female cast', 'romance', 'school', 'school life', 'seinen', 'sentimental drama', 'slice of life', 'stereotypes', 'suicide', 'supernatural', 'supernatural drama', 'tragedy', 'tr

In [19]:
get_rec_df("causal") |> inv(seen) |> inv(related) |> top(N) |> display

Rank,Title,Score,Type,Episodes,Episode Length,Start Date,Genres,Tags,Rating,Watch Probability,Relative Rating
1,Odd Taxi,10.4914,tv,13,24m,2021-04-06,"['Anthropomorphic', 'Award Winning', 'Mystery']","['adoption', 'age gap', 'amnesia', 'animal protagonists', 'animals', 'anthropomorphic', 'anthropomorphism', 'anti-hero', 'bar', 'beastmen', 'bullying', 'cars', 'comedy', 'crime', 'crime fiction', 'denpa', 'drama', 'ensemble cast', 'fugitive', 'gangs', 'heterosexual', 'idol', 'mafia', 'male protagonist', 'martial arts', 'mystery', 'new', 'noir', 'non-human protagonists', 'original work', 'philosophy', 'police', 'present', 'primarily adult cast', 'psychological', 'revenge', 'slice of life', 'slice of life drama', 'social media', 'suicide', 'supernatural drama', 'thriller', 'tragedy', 'twins', 'urban', 'video games', 'work', 'yakuza']",6.5563,0.0047332,0.865047
2,Top wo Nerae! Gunbuster,10.4649,ova,6,30m,1988-10-07,"['Action', 'Comedy', 'Drama', 'Mecha', 'Military', 'Sci-Fi', 'Space']","['achromatic', 'action', 'adventure', 'alien', 'aliens', 'astronomy', 'bullying', 'comedy', 'coming of age', 'cosmic horror', 'drama', 'ecchi', 'female protagonist', 'future', 'henshin', 'kaiju', 'love triangle', 'mecha', 'military', 'new', 'nudity', 'original work', 'outer space', 'parody', 'philosophy', 'piloted robot', 'primarily female cast', 'robot', 'school', 'sci fi', 'sci-fi', 'science fiction', 'science-fiction', 'seinen', 'shipboard', 'shounen', 'space', 'space opera', 'space travel', 'super robot', 'super robots', 'time manipulation', 'time skip', 'tomboy', 'tragedy', 'travel', 'war']",6.43367,0.0035682,0.912471
3,Maria-sama ga Miteru,10.4062,tv,13,24m,2004-01-08,"['Drama', 'Girls Love']","['all girls school', 'all-girls school', 'asia', 'based on a light novel', 'catholic school', 'coming of age', 'daily life', 'drama', 'earth', 'female protagonist', 'girls love', 'gl', 'high school', 'japan', 'lgbt themes', 'novel', 'ojou-sama', 'present', 'primarily female cast', 'romance', 'school', 'school club', 'school life', 'shoujo', 'shoujo ai', 'shoujo-ai', 'slice of life', 'slice of life drama', 'yuri']",5.49384,0.0031679,0.915869
4,Cyberpunk: Edgerunners,10.2293,ona,10,25m,2022-09-13,"['Action', 'Gore', 'Psychological', 'Sci-Fi']","['action', 'alternative world', 'american derived', 'anti-hero', 'artificial intelligence', 'based on a video game', 'bionic powers', 'body horror', 'cars', 'coming of age', 'contract killer / assassin', 'cosmic horror', 'crime', 'cyberpunk', 'cyborg', 'cyborgs', 'drama', 'drugs', 'dystopia', 'dystopian', 'ecchi', 'ensemble cast', 'excessive violence', 'game', 'gangs', 'gore', 'guns', 'hero of strong character', 'heroine of strong character', 'heterosexual', 'male protagonist', 'nudity', 'primarily adult cast', 'psychological', 'revenge', 'romance', 'sci fi', 'sci-fi', 'science fiction', 'science-fiction', 'software development', 'space', 'splatter', 'struggle to survive', 'terrorism', 'time manipulation', 'time skip', 'tragedy', 'urban', 'violence']",6.56991,0.00500535,0.884835
5,Kanon (2006),10.1844,tv,24,24m,2006-10-06,"['Drama', 'Romance', 'Slice of Life', 'Supernatural']","['age regression', 'amnesia', 'asia', 'based on a visual novel', 'comedy', 'coming of age', 'contemporary fantasy', 'demons', 'drama', 'earth', 'erotic game', 'family life', 'fantasy', 'female harem', 'game', 'ghost', 'harem', 'heterosexual', 'high school', 'japan', 'kuudere', 'male protagonist', 'moe', 'present', 'primarily female cast', 'romance', 'school', 'school life', 'seinen', 'sentimental drama', 'slice of life', 'stereotypes', 'suicide', 'supernatural', 'supernatural drama', 'tragedy', 'transfer students', 'tsundere', 'twisted story', 'visual novel']",7.03501,0.00335769,1.07615
6,Spy x Family,10.136,tv,12,24m,2022-04-09,"['Action', 'Childcare', 'Comedy', 'Shounen']","['action', 'action comedy', 'adoption', 'assassin', 'assassins', 'based on a manga', 'bullying', 'childcare', 'comedy', 'conspiracy', 'contemporary fantasy', 'contract k

In [20]:
get_rec_df("temporal") |> inv(seen) |> inv(related) |> top(N) |> display

Rank,Title,Score,Type,Episodes,Episode Length,Start Date,Genres,Tags,Rating,Watch Probability,Relative Rating
1,Kanon (2006),6.89927,tv,24,24m,2006-10-06,"['Drama', 'Romance', 'Slice of Life', 'Supernatural']","['age regression', 'amnesia', 'asia', 'based on a visual novel', 'comedy', 'coming of age', 'contemporary fantasy', 'demons', 'drama', 'earth', 'erotic game', 'family life', 'fantasy', 'female harem', 'game', 'ghost', 'harem', 'heterosexual', 'high school', 'japan', 'kuudere', 'male protagonist', 'moe', 'present', 'primarily female cast', 'romance', 'school', 'school life', 'seinen', 'sentimental drama', 'slice of life', 'stereotypes', 'suicide', 'supernatural', 'supernatural drama', 'tragedy', 'transfer students', 'tsundere', 'twisted story', 'visual novel']",7.35393,0.00221708,1.14784
2,Top wo Nerae! Gunbuster,6.80248,ova,6,30m,1988-10-07,"['Action', 'Comedy', 'Drama', 'Mecha', 'Military', 'Sci-Fi', 'Space']","['achromatic', 'action', 'adventure', 'alien', 'aliens', 'astronomy', 'bullying', 'comedy', 'coming of age', 'cosmic horror', 'drama', 'ecchi', 'female protagonist', 'future', 'henshin', 'kaiju', 'love triangle', 'mecha', 'military', 'new', 'nudity', 'original work', 'outer space', 'parody', 'philosophy', 'piloted robot', 'primarily female cast', 'robot', 'school', 'sci fi', 'sci-fi', 'science fiction', 'science-fiction', 'seinen', 'shipboard', 'shounen', 'space', 'space opera', 'space travel', 'super robot', 'super robots', 'time manipulation', 'time skip', 'tomboy', 'tragedy', 'travel', 'war']",6.50802,0.00237699,0.922977
3,Odd Taxi,6.73655,tv,13,24m,2021-04-06,"['Anthropomorphic', 'Award Winning', 'Mystery']","['adoption', 'age gap', 'amnesia', 'animal protagonists', 'animals', 'anthropomorphic', 'anthropomorphism', 'anti-hero', 'bar', 'beastmen', 'bullying', 'cars', 'comedy', 'crime', 'crime fiction', 'denpa', 'drama', 'ensemble cast', 'fugitive', 'gangs', 'heterosexual', 'idol', 'mafia', 'male protagonist', 'martial arts', 'mystery', 'new', 'noir', 'non-human protagonists', 'original work', 'philosophy', 'police', 'present', 'primarily adult cast', 'psychological', 'revenge', 'slice of life', 'slice of life drama', 'social media', 'suicide', 'supernatural drama', 'thriller', 'tragedy', 'twins', 'urban', 'video games', 'work', 'yakuza']",6.7614,0.00517262,0.89775
4,Cyberpunk: Edgerunners,6.71654,ona,10,25m,2022-09-13,"['Action', 'Gore', 'Psychological', 'Sci-Fi']","['action', 'alternative world', 'american derived', 'anti-hero', 'artificial intelligence', 'based on a video game', 'bionic powers', 'body horror', 'cars', 'coming of age', 'contract killer / assassin', 'cosmic horror', 'crime', 'cyberpunk', 'cyborg', 'cyborgs', 'drama', 'drugs', 'dystopia', 'dystopian', 'ecchi', 'ensemble cast', 'excessive violence', 'game', 'gangs', 'gore', 'guns', 'hero of strong character', 'heroine of strong character', 'heterosexual', 'male protagonist', 'nudity', 'primarily adult cast', 'psychological', 'revenge', 'romance', 'sci fi', 'sci-fi', 'science fiction', 'science-fiction', 'software development', 'space', 'splatter', 'struggle to survive', 'terrorism', 'time manipulation', 'time skip', 'tragedy', 'urban', 'violence']",6.46554,0.0175546,0.872433
5,Nana,6.69533,tv,47,23m,2006-04-05,"['Adult Cast', 'Drama', 'Love Polygon', 'Music', 'Romance', 'Shoujo', 'Slice of Life']","['adult cast', 'adult couples', 'age gap', 'angst', 'asia', 'band', 'based on a manga', 'college', 'comedy', 'coming of age', 'drama', 'earth', 'female protagonist', 'gyaru', 'heroine of strong character', 'heterosexual', 'japan', 'josei', 'kuudere', 'love polygon', 'love triangle', 'mahjong', 'manga', 'mature romance', 'music', 'musical band', 'netorare', 'noir', 'plot continuity', 'pregnancy', 'present', 'primarily adult cast', 'rock music', 'romance', 'romantic drama', 'shoujo', 'showbiz', 'slice of life', 'tokyo', 'tragedy', 'verbal comedy']",7.45182,0.00237619,1.0146
6,Spy x Family,6.67384,tv,12,24m,2022-04-09,"['Action', 'Childcare', 'Comedy', 